In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd

/Users/xinyutian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/xinyutian/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('train.csv')
use_data = data[['id', 
                      'name', 
                      'description', 
                      'host_verifications', 
                      'neighbourhood_cleansed', 
                      'neighbourhood_group_cleansed',
                      'property_type',
                      'room_type',
                      'bathrooms_text',
                      'amenities',
                      'price']]

train_data, test_data = train_test_split(use_data, test_size=0.2, random_state=42)
train_data = TabularDataset(train_data) 
test_data = TabularDataset(test_data)

predictor = TabularPredictor(label='price').fit(train_data=train_data)
predictions = predictor.predict(test_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20231203_061120"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231203_061120"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.7
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 23.1.0: Mon Oct  9 21:27:24 PDT 

In [4]:
# use the best model to predict based on test.csv 
pred_data = pd.read_csv('test.csv')
pred_data = pred_data[['id', 
                      'name', 
                      'description', 
                      'host_verifications', 
                      'neighbourhood_cleansed', 
                      'neighbourhood_group_cleansed',
                      'property_type',
                      'room_type',
                      'bathrooms_text',
                      'amenities']]
pred_data = TabularDataset(pred_data)
predictions = predictor.predict(pred_data)

# save the predictions to a csv file that only contains the id and the price columns
predictions.to_csv('submission.csv', index=False)
# add an ID column to the predictions
submission = pd.read_csv('submission.csv')
submission.insert(0, 'id', range(0, len(submission)))
submission.to_csv('submission.csv', index=False)

In [5]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.560052,0.569579,accuracy,0.852850,0.324258,214.108756,0.001869,0.000447,0.283991,2,True,9
1,CatBoost,0.549045,0.551780,accuracy,0.235078,0.124845,143.429339,0.235078,0.124845,143.429339,1,True,4
2,XGBoost,0.549045,0.556634,accuracy,0.350018,0.104317,46.178284,0.350018,0.104317,46.178284,1,True,7
3,RandomForestGini,0.531887,0.531553,accuracy,0.188234,0.072511,6.478947,0.188234,0.072511,6.478947,1,True,2
4,RandomForestEntr,0.522499,0.541262,accuracy,0.173235,0.072896,6.264850,0.173235,0.072896,6.264850,1,True,3
5,ExtraTreesGini,0.521204,0.528317,accuracy,0.154161,0.071558,6.359418,0.154161,0.071558,6.359418,1,True,5
6,ExtraTreesEntr,0.516025,0.524272,accuracy,0.212898,0.072794,5.896620,0.212898,0.072794,5.896620,1,True,6
7,NeuralNetFastAI,0.501457,0.498382,accuracy,0.041937,0.018176,10.899491,0.041937,0.018176,10.899491,1,True,1
8,NeuralNetTorch,0.500809,0.521036,accuracy,0.052987,0.021855,18.320521,0.052987,0.021855,18.320521,1,True,8


In [6]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 10 features using 3089 rows with 5 shuffle sets...
	127.9s	= Expected runtime (25.58s per shuffle set)
	107.13s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
description,0.057559,0.006850,2.361957e-05,5,0.071663,0.043455
name,0.055422,0.005438,1.098311e-05,5,0.066620,0.044225
amenities,0.055422,0.002236,3.171474e-07,5,0.060026,0.050819
bathrooms_text,0.051602,0.005492,1.516627e-05,5,0.062910,0.040294
property_type,0.038459,0.003375,7.045760e-06,5,0.045409,0.031510
neighbourhood_cleansed,0.032373,0.004270,3.550346e-05,5,0.041166,0.023580
room_type,0.002007,0.001625,2.538052e-02,5,0.005353,-0.001339
host_verifications,0.000842,0.002130,2.134312e-01,5,0.005228,-0.003544
id,-0.001360,0.001154,9.710683e-01,5,0.001016,-0.003735
neighbourhood_group_cleansed,-0.002137,0.002003,9.621981e-01,5,0.001989,-0.006262
